### 1. 정규 표현식(Regular Expression, Regex)

> 일정한 규칙을 가진 문자열의 집합을 표현하고 처리하는데 사용되며,ㅡ 특히 문자열의 검색 및 치환에 활용됨

> re : 정규식 사용을 위한 패키지

> (참고자료) : w3schools.com : RegEX


In [1]:
!pip install re

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [2]:
# 원하는 형태 : 정규식
# . (ca.e): 하나의 문자를 의미 > care, cafe, case (o) | caffe (x)
# ^ (^de): 문자열의 시작 > desk, destination (o) | fade(x)
# $ (se$): 문자열의 끝 > case, base (o) | face(x)


import re
p = re.compile("ca.e")  # re.compile : 파이썬이 사용할 수 있는 정규식 객체로 컴파일 해주는 기능

def print_match(m):
    if m: 
        print("m.string:", m.string) # 입력받은 문자열
        print("m.group():", m.group()) # 일치하는 문자열 반환
        print("m.start():", m.start()) # 일치하는 문자열의 시작 index
        print("m.end():", m.end()) # 일치하는 문자열의 끝 index
        print("m.span():", m.span()) # 일치하는 문자열의 시작과 끝 index
        print()
    else:
        print("일치하지 않음")
        print()        
        
m = p.match("careless") # match : 주어진 문자열의 처음부터 일치하는 지 확인
print(m)
print_match(m)
print("*"*100)

m = p.match("good care") # match : 주어진 문자열의 처음부터 일치하는 지 확인
print(m)
print_match(m)
print("*"*100)

m = p.search("good care") # search : 주어진 문자열중에 일치하는게 있는 지 확인
print(m)
print_match(m)
print("*"*100)

lst = p.findall("good care cafe") # findall : 일치하는 모든 것을 리스트형태로 반환
print(lst)


# 1. p = re.compile("원하는 형태") : 정규식 객체로 컴파일
# 2. m = p.match("비교할 문자열") : 주어진 문자열의 처음부터 일치하는 지 확인
# 3. m = p.search("비교할 문자열") : 주어진 문자열중에 일치하는게 있는 지 확인
# 4. lst = p.findall("비교할 문자열") : 일치하는 모든 것을 리스트형태로 반환



<re.Match object; span=(0, 4), match='care'>
m.string: careless
m.group(): care
m.start(): 0
m.end(): 4
m.span(): (0, 4)

****************************************************************************************************
None
일치하지 않음

****************************************************************************************************
<re.Match object; span=(5, 9), match='care'>
m.string: good care
m.group(): care
m.start(): 5
m.end(): 9
m.span(): (5, 9)

****************************************************************************************************
['care', 'cafe']


###  2. 쿠팡 웹스크래핑 

#### 2.1 쿠팡 웹스크래핑

In [ ]:
### 쿠팡 웹스크래핑
import requests
import re
from bs4 import BeautifulSoup

url = "https://www.coupang.com/np/search?q=%EB%85%B8%ED%8A%B8%EB%B6%81&channel=user&component=&eventCategory=SRP&trcid=&traid=&sorter=scoreDesc&minPrice=&maxPrice=&priceRange=&filterType=&listSize=36&filter=&isPriceRange=false&brand=&offerCondition=&rating=0&page=1&rocketAll=false&searchIndexingToken=1=4&backgroundColor="
# res = requests.get(url)

#"User-Agent" 필요
headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.128 Safari/537.36"}
res = requests.get(url, headers=headers)
res.raise_for_status()
soup = BeautifulSoup(res.text, "lxml") 

items = soup.find_all("li", attrs={"class":re.compile("^search-product")})
print(items[0].find("div", attrs={"class":"name"}).get_text())
print("*"*100)

for item in items:
    
    # 상품명, 가격 조회
    name = item.find("div", attrs={"class":"name"}).get_text() # 제품명
    price = item.find("strong", attrs={"class":"price-value"}) # 가격
    price = price.get_text()

    # 평점 조회
    rate = item.find("em", attrs={"class":"rating"}) # 평점
    if rate:
        rate = rate.get_text()
    else:
        continue  
    
#     rate = item.find("em", attrs={"class":"rating"}) # 평점
#     rate = rate.get_text()
    
    # 평점 수 조회            
    rate_cnt = item.find("span", attrs={"class":"rating-total-count"}) # 평점 수
    if rate_cnt:
        rate_cnt = rate_cnt.get_text() # 예 : (26)
        rate_cnt = rate_cnt[1:-1]
    else:
        continue

#     rate_cnt = item.find("span", attrs={"class":"rating-total-count"}) # 평점 수
#     rate_cnt = rate_cnt.get_text() # 예 : (26)
#     rate_cnt = rate_cnt[1:-1]

    # 리뷰 200개 이상,  평점 4.5이상되는 것만 조회
    if float(rate) >= 4.5 and int(rate_cnt) >= 100:
        print(name, price, rate, rate_cnt)    
        print("-"*100)        

In [3]:
### 쿠팡 웹스크래핑
import requests
import re
from bs4 import BeautifulSoup

url = "https://www.coupang.com/np/search?component=&q=%EB%85%B8%ED%8A%B8%EB%B6%81&channel=user"
# res = requests.get(url)

#"User-Agent" 필요
headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"}
res = requests.get(url, headers=headers)
res.raise_for_status() # 제대로 작동이 안되면 에러 발생시킴
soup = BeautifulSoup(res.text, "lxml")

items = soup.find_all("li", attrs={"class":re.compile("^search-product")})
print(items[0].find("div", attrs={"class":"name"}).get_text())
print("*"*100)

for item in items:
    
    # 상품명, 가격 조회
    name = item.find("div", attrs={"class":"name"}).get_text() # 제품명
    price = item.find("strong", attrs={"class":"price-value"}) # 가격
    price = price.get_text()

    # 평점 조회
    rate = item.find("em", attrs={"class":"rating"}) # 평점
    if rate:
        rate = rate.get_text() # 평점이 없을때를 대비해서 에러가 안나도록 방지
    else:
        continue  
    
#     rate = item.find("em", attrs={"class":"rating"}) # 평점
#     rate = rate.get_text()
    
    # 평점 수 조회            
    rate_cnt = item.find("span", attrs={"class":"rating-total-count"}) # 평점 수
    if rate_cnt:
        rate_cnt = rate_cnt.get_text() # 예 : (26)
        rate_cnt = rate_cnt[1:-1]
    else:
        continue

#     rate_cnt = item.find("span", attrs={"class":"rating-total-count"}) # 평점 수
#     rate_cnt = rate_cnt.get_text() # 예 : (26)
#     rate_cnt = rate_cnt[1:-1]

    # 리뷰 200개 이상,  평점 4.5이상되는 것만 조회
    if float(rate) >= 4.5 and int(rate_cnt) >= 100:
        print(name, price, rate, rate_cnt)    
        print("-"*100)        

삼성 아티브북9 Lite NT910S3T i3 8G SSD256 Win10
****************************************************************************************************
레노버 아이디어패드 Platinum Grey 노트북 S150-11 81VT (Intel Celeron N4020 29.5cm WIN10 Home S), 윈도우 포함, 64GB, 4GB 289,000 4.5 142
----------------------------------------------------------------------------------------------------
이태원클라쓰북 그램스타일 노트북 풀패키지미개봉 NB141LTN41 8세대 14 IPS FHD 윈10탑재, 화이트, NB133LTN40 [SDD256G+32G] 419,000 4.5 246
----------------------------------------------------------------------------------------------------
삼성전자 플러스 퓨어 화이트 노트북 NT350XCR-AD3AW (i3-10110U), 포함, 256GB, 8GB 798,000 5.0 511
----------------------------------------------------------------------------------------------------
삼성전자 플러스2 퓨어화이트 노트북 NT550XDA-K78AW (i7-1165G7 39.6cm WIN10 Home), 포함, 256GB, 16GB 1,249,000 5.0 899
----------------------------------------------------------------------------------------------------
삼성전자 노트북9 METAL NT901X5L 가볍고 슬림한 1.29kg 코어i5 윈10 탑

#### 2.2 광고 제품, 가격없는 제품, 평점 및 평점 수 없는 제품, 애플 제품 제거 

In [4]:
### 쿠팡 웹스크래핑
import requests
import re
from bs4 import BeautifulSoup

url = "https://www.coupang.com/np/search?q=%EB%85%B8%ED%8A%B8%EB%B6%81&channel=user&component=&eventCategory=SRP&trcid=&traid=&sorter=scoreDesc&minPrice=&maxPrice=&priceRange=&filterType=&listSize=36&filter=&isPriceRange=false&brand=&offerCondition=&rating=0&page=1&rocketAll=false&searchIndexingToken=1=4&backgroundColor="
# res = requests.get(url)

#"User-Agent" 필요
headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.128 Safari/537.36"}
res = requests.get(url, headers=headers)
res.raise_for_status()
soup = BeautifulSoup(res.text, "lxml") 

items = soup.find_all("li", attrs={"class":re.compile("^search-product")})
print(items[0].find("div", attrs={"class":"name"}).get_text())
print("*"*100)

for item in items:
    
    # 광고 제품은 제외
    ad_badge = item.find("span", attrs={"class" : "ad-badge-text"})
    if ad_badge:
        print("-"*100)
        print(" <광고 상품 제외합니다.> ")
        continue
    
    name = item.find("div", attrs={"class":"name"}).get_text() # 제품명
    price = item.find("strong", attrs={"class":"price-value"}) # 가격
    if price: # 가격 있으면 get_text()
        price = price.get_text()
    else:
        print("-"*100)
        print(" <가격없는 상품 제외합니다.> ")
        continue  
        
    rate = item.find("em", attrs={"class":"rating"})# 평점
    if rate:
        rate = rate.get_text()
    else:
        print("-"*100)
        print(" <평점없는 상품 제외합니다.> ")
        continue  
        
    rate_cnt = item.find("span", attrs={"class":"rating-total-count"}) # 평점 수
    if rate_cnt:
        rate_cnt = rate_cnt.get_text() # 예 : (26)
        rate_cnt = rate_cnt[1:-1]
    else:
        print("-"*100)
        print(" <평점수 없는 상품 제외합니다.> ")
        continue
        
    # 애플 제품 제외
    if "Apple" in name:
        print("-"*100)
        print(" <Apple 상품 제외합니다.> ")
        continue
        
    # 리뷰 200개 이상,  평점 4.5이상되는 것만 조회
    if float(rate) >= 4.5 and int(rate_cnt) >= 100:
        print(name, price, rate, rate_cnt)
        print("-"*100)        

삼성 아티브북9 Lite NT910S3T i3 8G SSD256 Win10
****************************************************************************************************
----------------------------------------------------------------------------------------------------
 <광고 상품 제외합니다.> 
----------------------------------------------------------------------------------------------------
 <평점없는 상품 제외합니다.> 
레노버 아이디어패드 Platinum Grey 노트북 S150-11 81VT (Intel Celeron N4020 29.5cm WIN10 Home S), 윈도우 포함, 64GB, 4GB 289,000 4.5 142
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
 <광고 상품 제외합니다.> 
----------------------------------------------------------------------------------------------------
 <광고 상품 제외합니다.> 
삼성전자 플러스 퓨어 화이트 노트북 NT350XCR-AD3AW (i3-10110U), 포함, 256GB, 8GB 768,000 5.0 511
------------------------------------------------------------------------------------

#### 2.3 여러 페이지에서 정보 가져오기

In [7]:
import requests
import re
from bs4 import BeautifulSoup

headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.128 Safari/537.36"}

for i in range(1, 6):
    print("페이지 : ", i)
    url = "https://www.coupang.com/np/search?q=%EB%85%B8%ED%8A%B8%EB%B6%81&channel=user&component=&eventCategory=SRP&trcid=&traid=&sorter=scoreDesc&minPrice=&maxPrice=&priceRange=&filterType=&listSize=36&filter=&isPriceRange=false&brand=&offerCondition=&rating=0&page={}&rocketAll=false&searchIndexingToken=1=4&backgroundColor=".format(i)

    res = requests.get(url, headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml") 

    items = soup.find_all("li", attrs={"class":re.compile("^search-product")})
    
    for item in items:

        # 광고 제품은 제외
        ad_badge = item. find("span", attrs={"class" : "ad-badge-text"})
        if ad_badge:
            #print(" <광고 상품 제외합니다.> ")
            continue

        name = item.find("div", attrs={"class":"name"}).get_text() # 제품명
        price = item.find("strong", attrs={"class":"price-value"}) # 가격
        if price:
            price = price.get_text()
        else:
            #print(" <가격없는 상품 제외합니다.> ")
            continue  

        rate = item.find("em", attrs={"class":"rating"})# 평점
        if rate:
            rate = rate.get_text()
        else:
            #print(" <평점없는 상품 제외합니다.> ")
            continue  

        rate_cnt = item.find("span", attrs={"class":"rating-total-count"}) # 평점 수
        if rate_cnt:
            rate_cnt = rate_cnt.get_text() # 예 : (26)
            rate_cnt = rate_cnt[1:-1]
        else:
            #print(" <평점수 없는 상품 제외합니다.> ")
            continue

        # 애플 제품 제외
        if "Apple" in name:
            #print(" <Apple 상품 제외합니다.> ")
            continue
        
        link = item.find("a", attrs={"class":"search-product-link"})["href"]
        
        # 리뷰 200개 이상,  평점 4.5이상되는 것만 조회
        if float(rate) >= 4.5 and int(rate_cnt) >= 200:
            print(f"제품명 : {name}")
            print(f"가격 : {price}")
            print(f"평점 : {rate}점 ({rate_cnt}개)")
            print("바로가기 : {}".format("https://www.coupang.com"+link))
            print("-"*100)
            


페이지 :  1
제품명 : 삼성전자 플러스 퓨어 화이트 노트북 NT350XCR-AD3AW (i3-10110U), 포함, 256GB, 8GB
가격 : 768,000
평점 : 5.0점 (511개)
바로가기 : https://www.coupang.com/vp/products/4698318558?itemId=5914282411&vendorItemId=73212343669
----------------------------------------------------------------------------------------------------
제품명 : 삼성전자 플러스2 퓨어화이트 노트북 NT550XDA-K78AW (i7-1165G7 39.6cm WIN10 Home), 포함, 256GB, 16GB
가격 : 1,249,000
평점 : 5.0점 (899개)
바로가기 : https://www.coupang.com/vp/products/4692708274?itemId=5899936424&vendorItemId=73198049954
----------------------------------------------------------------------------------------------------
제품명 : 삼성전자 노트북9 METAL NT901X5L 가볍고 슬림한 1.29kg 코어i5 윈10 탑재, 포함, SSD 256GB, 8GB
가격 : 659,000
평점 : 4.5점 (548개)
바로가기 : https://www.coupang.com/vp/products/4368907108?itemId=5147093216&vendorItemId=72456426781
----------------------------------------------------------------------------------------------------
제품명 : 레노버 노트북 V14-ARE 82DQ003AKR (R5-4500U 35.56cm), 윈도우 미포함, 256GB, 4

### 3. 네이버 코스피 정보 가져오기 및 CSV 저장

In [8]:
import csv
import requests
from bs4 import BeautifulSoup
import pandas as pd 

headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.128 Safari/537.36"}

url = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page="

filename = "네이버코스피정보.csv"
f = open(filename, "w", encoding="utf-8-sig", newline="") # newline="" 줄바꿈을 없애줌 , utf-8-sig 엑셀에서 한글이 깨지는 경우 대처방안
writer = csv.writer(f)

title = "N	종목명	현재가	전일비	등락률	액면가	시가총액	상장주식수	외국인비율	거래량	PER	ROE".split("\t")
print(title)
writer.writerow(title)

for page in range(1, 5):
    res = requests.get(url + str(page), headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml") 
    
    data_rows = soup.find("table", attrs={"class":"type_2"}).find("tbody").find_all("tr")
    for row in data_rows:
        columns = row.find_all("td")
        if len(columns) <= 1: # 의미없는 데이터는 skip
            continue
        data = [column.get_text().strip() for column in columns]
        writer.writerow(data) # ()안에는 리스트 형태로

        
# CSV -> DataFrame        
df_csv = pd.read_csv("네이버코스피정보.csv", header = 0, index_col = False)
display(df_csv)


['N', '종목명', '현재가', '전일비', '등락률', '액면가', '시가총액', '상장주식수', '외국인비율', '거래량', 'PER', 'ROE']


,N,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE
0,1,삼성전자,"79,500",100,-0.13%,100,"4,745,977","5,969,783",53.75,"15,074,106",20.70,9.99
1,2,SK하이닉스,"122,500","1,500",+1.24%,"5,000","891,803","728,002",48.95,"4,452,141",18.75,9.53
2,3,LG화학,"903,000","22,000",+2.50%,"5,000","637,449","70,592",45.62,"290,418",39.31,2.93
3,4,삼성전자우,"73,200",400,-0.54%,100,"602,353","822,887",76.89,"1,745,843",19.06,NaN
4,5,삼성바이오로직스,"891,000","3,000",-0.34%,"2,500","589,530","66,165",10.26,"211,841",244.65,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
169,170,롯데정밀화학,"60,000",700,-1.15%,"5,000","15,480","25,800",22.33,"138,463",7.79,12.65
170,171,KODEX 레버리지,"27,155",195,-0.71%,0,"15,424","56,800",2.30,"17,044,513",NaN,NaN
171,172,KODEX 2차전지산업,"18,985",200,+1.06%,0,"15,017","79,100",0.02,"679,000",NaN,NaN
172,173,에스엘,"31,100","1,400",+4.71%,500,"14,991","48,203",14.96,"1,527,142",22.83,4.74


### 4. RPA

Robotic process automation (RPA)

1. 로보틱 처리 자동화, 로보틱 프로세스 자동화, 로보틱 프로세스 오토메이션 으로 불림

2. 소프트웨어 봇 또는 인공지능(AI) 워커(worker)의 개념에 기반을 둔, 최근에 각광받는 비즈니스 프로세스 자동화 기술

3. RPA는 사용자 인터페이스를 활용하고, 모든 S/W 시스템에서도 실행할 수 있는 소프트웨어를 통한 규칙기반 프로세스의 자동화

4. 매크로와의 차이점
> RPA는 매크로와 달리 다양한 S/W에 걸쳐 작동가능하며, 중앙집중관리가 가능함. 또한 인공지능과 결합하여 많은 프로세스 자동화에 활용가능

5. RPA의 핵심특징
> 비침투성 : 시스템 통합이 필요없음
> 개발용이성 : 향후 비즈니스 인력이 개발 및 유지보수 가능

6. RPA의 도입효과
> 휴먼 에러 감소
> 업무 생산성 향상
> 인력 관리 효율성 증대

7. RPA 예제 : https://www.dropbox.com/s/gv1470lylp9sgre/RPA_example.PNG

### 5. 나만의 비서 만들기

In [ ]:
# 1. 네이버에서 오늘 서울의 날씨 정보 검색
# 2. 헤드라인 뉴스 3건 검색
# 3. IT 뉴스 3건 검색 
# 4. 해커스 어학원 홈페이지에서 오늘의 영어회화 지문 검색

# [오늘의 날씨]
# 맑음, 어제보다 4˚ 높아요
# 현재 20℃  (최저 14˚ / 최고 25˚)
# 오전 강수확률 0% / 오후 강수확률 0%

# 미세먼지 33㎍/㎥보통
# 초미세먼지 13㎍/㎥좋음

# [헤드라인 뉴스]
# 1, 국내 주요 4종 변이 바이러스 247건 추가 확인…“발생 지역 확대 우려”
#   (링크 : https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=056&aid=0011046422 )
# 2, 관평원 시세차익 뭇매 관세청, 부동산 시세 자료 토대 적극 방어
#   (링크 : https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=421&aid=0005358492 )
# 3, 비트코인 5500만원대로 오락가락…머스크 환경파괴설 역공?
#   (링크 : https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=105&oid=016&aid=0001836957 )

# [IT 뉴스]
# 1, 크래프톤, 1분기 매출 4610억원… 해외 비중 94%
#   (링크 : https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=105&sid2=230https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=005&aid=0001440212 )
# 2, 대기오염 주범 이산화질소, 파킨슨병 발병 위험 41% ↑
#   (링크 : https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=105&sid2=230https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=056&aid=0011046595 )
# 3, SKT 티맵모빌리티, 물류 스타트업 '와이피엘' 790억원에 인수
#   (링크 : https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=105&sid2=230https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=003&aid=0010505152 )

# [오늘의 영어회화]
# (한글 지문)
# Bertha: 우리는 전에 만난 적이 있지 않나요?
# Rob: 그랬던 것 같네요. 지난 여름 Communications Infrastructure 회의에 있었나요? 우리가 그 회의를 후원했어요.
# Bertha: 네, 이제 기억이 나네요. 저는 그곳에 있었어요. 당신은 Smart Word Software Solutions에 다니죠.
# Rob: 맞아요, 저는 Rob Parnell입니다.

# (영어 지문)
# Bertha: Haven't we met before?
# Rob: We may have. Were you at the Communications Infrastructure Convention last summer? We sponsored that convention.
# Bertha: Yes, I remember now. I was there. You’re with Smart Word Software Solutions.
# Rob: Right, I’m Rob Parnell.


In [10]:
import requests
import re
from bs4 import BeautifulSoup

def create_soup(url):
    headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
    res = requests.get(url, headers=headers)
    res.raise_for_status()    

    soup = BeautifulSoup(res.text, "lxml")
    return soup

def print_news(index, title, link):
    print("{}, {}".format(index + 1, title))
    print("  (링크 : {} )".format(link))

def scrap_weather():
    print("[오늘의 날씨]")
    url = "https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EC%84%9C%EC%9A%B8+%EB%82%A0%EC%94%A8"
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")
    cast = soup.find("p", attrs={"class":"cast_txt"}).get_text()
    curr_temp = soup.find("p", attrs={"class":"info_temperature"}).get_text().replace("도씨", "")
    min_temp = soup.find("span", attrs={"class":"min"}).get_text()
    max_temp = soup.find("span", attrs={"class":"max"}).get_text()
    morning_rain_rate = soup.find("span", attrs={"class":"point_time morning"}).get_text().strip()
    afternoon_rain_rate = soup.find("span", attrs={"class":"point_time afternoon"}).get_text().strip()    
    
    dust = soup.find("dl", attrs={"class":"indicator"})
    pm10 = dust.find_all("dd")[0].get_text() # 미세먼지
    pm25 = dust.find_all("dd")[1].get_text() # 초미세먼지
    
    # 출력
    print(cast)
    print("현재 {} (최저 {} / 최고 {})".format(curr_temp, min_temp, max_temp))
    print("오전 {} / 오후 {}".format(morning_rain_rate, afternoon_rain_rate))
    print()
    print("미세먼지 {}".format(pm10))
    print("초미세먼지 {}".format(pm25))
    print()
    
def scrap_headline_news():
    print("[헤드라인 뉴스]")
    url = "https://news.naver.com"
    soup = create_soup(url)
    news_list = soup.find("ul", attrs={"class":"hdline_article_list"}).find_all("li", limit = 3)
    for index, news in enumerate(news_list):
        title = news.find("a").get_text().strip() # 띄어씌기 없애려고 strip() 사용.
        link = url + news.find("a")["href"]
        print("{}, {}".format(index + 1, title))
        print("  (링크 : {} )".format(link))
    print()
    
def scrap_IT_news():
    print("[IT 뉴스]")
    url = "https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=105&sid2=230"
    soup = create_soup(url)
    news_list = soup.find("ul", attrs={"class":"type06_headline"}).find_all("li", limit = 3)
    for index, news in enumerate(news_list):
        a_idx = 0
        img = news.find("img")
        if img:
            a_idx = 1 # img 태그가 있으면 1번째, a 태그의 정보를 활용
            
        title = news.find_all("a")[a_idx].get_text().strip()
        link = url + news.find_all("a")[a_idx]["href"]
        print_news(index, title, link)
    print()
    
def scrap_english():
    print("[오늘의 영어회화]")
    url = "https://www.hackers.co.kr/?c=s_eng/eng_contents/I_others_english&keywd=freelec_english_otherenglish&logger_kw=freelec_english_otherenglish&source=logger_kw"
    soup = create_soup(url)
    sentences = soup.find_all("div", attrs={"id":re.compile("^conv_kor_t")})
    print("(한글 지문)")
    for i, sentence in enumerate(sentences[:len(sentences)//2]):# 처음부터 절반까지
        print("{}. {}".format(i+1, sentence.get_text().strip()))          
    print()
    
    print("(영어 지문)")
    for i, sentence in enumerate(sentences[len(sentences)//2:]):# 절반부터 끝까지
        print("{}. {}".format(i+1, sentence.get_text().strip()))        
    print()
    
if __name__ == "__main__":
    scrap_weather()
    scrap_headline_news()
    scrap_IT_news()
    scrap_english()
    

[오늘의 날씨]
흐림, 어제보다 5˚ 낮아요
현재 21℃  (최저 17˚ / 최고 23˚)
오전 강수확률 30% / 오후 강수확률 80%

미세먼지 27㎍/㎥좋음
초미세먼지 12㎍/㎥좋음

[헤드라인 뉴스]
1, '버닝썬 경찰총장' 윤규근 총경 2심 일부 유죄 인정
  (링크 : https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=214&aid=0001119891 )
2, '국내 AZ백신 접종자 대상 교차접종 임상시험 나선다'[모멘트]
  (링크 : https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=001&aid=0012405501 )
3, '성장과 공정' vs '공정과 상식'…이재명?윤석열 각축
  (링크 : https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=002&aid=0002189864 )

[IT 뉴스]
1, "키즈 콘텐츠 키운다"…LGU+, 몬스터스튜디오에 지분투자
  (링크 : https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=105&sid2=230https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=015&aid=0004549349 )
2, 자동차관리 종합 서비스 플랫폼 ‘프롬카’, 정비소 연결부터 매매까지 다양한 서비스 제공
  (링크 : https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=105&sid2=230https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=014&aid=0004642391 )
3, SKT-카카오VX, 메타버스 라이브 골프중계 첫선
  (링크